152120211052 - Umut Öztürk


152120211057 - Atakan Berber

-------Kütüphaneler---------

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from IPython.core.display import display, HTML


-------Dosya okuma-------

In [ ]:
df = pd.read_csv("Heart_Disease_Prediction.csv")

----İlk 5-----

In [ ]:
df.head()

----Verilerin açıklaması----

In [ ]:
df.describe()

In [ ]:
df.describe().T

-----Veri seti hakkında bilgi--------

In [ ]:
df.info()

-----Veri türleri-----

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.isnull()


In [ ]:
df.isnull().sum()

------Belirli kategorilerdeki değerlerin sayısını gösteren çubuk grafikler------

In [ ]:
cats = ["Sex", "Chest pain type", "FBS over 120", "EKG results", "Exercise angina"] + df.columns[-4:].tolist()

In [ ]:
index = 0
for i in range(3):
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 6))
    for j in range(3):
        counts = df[cats[index]].value_counts()
        counts.plot(kind="bar", ax=axes[j])
        for container in axes[j].containers:
            axes[j].bar_label(container)
        axes[j].set_yticklabels(())
        axes[j].set_ylabel("")
        axes[j].set_xlabel("")
        axes[j].set_title(cats[index])
        index += 1
    plt.tight_layout()
    plt.show()

-----Belirli kategoriler ile kalp hastalığı varlığı arasındaki ilişkiyi gösteren ısı haritaları (heatmaps)-----

In [ ]:
crosstabs = []
titles = []

for i in cats[:-1]:
    crosstab = pd.crosstab(df[i], df[df.columns[-1]])
    crosstabs += [crosstab]
    titles += ["{} vs Heart Disease presence".format(i)]

In [ ]:
index = 0

for i in range(2):
    fig, axes = plt.subplots(ncols=4, figsize=(15, 6))
    for j in range(4):
        sns.heatmap(crosstabs[index], annot=True, fmt='d', ax=axes[j])
        axes[j].set_title(titles[index])
        index += 1
    plt.tight_layout()
    plt.show()

Chest Paint Type

1: Typical angina (Tipik anjina)

Bu tip ağrı, genellikle fiziksel efor veya duygusal stres ile tetiklenir ve dinlenme veya nitrogliserin ile hafifler. Kalp ile ilgili bir sorun olduğuna işaret eder.

2: Atypical angina (Atipik anjina)

Bu tür ağrı, tipik anjinadan farklıdır ve eforla ilişkili olmayabilir. Farklı bir karakteristiğe sahip olabilir ve dinlenme veya nitrogliserin ile hafiflemeyebilir.

3: Non-anginal pain (Anjina dışı ağrı)

Bu tip ağrı, kalple ilgili olmayan ağrıları ifade eder. Örneğin, sindirim sistemi veya kas-iskelet sistemi kaynaklı ağrılar olabilir.

4: Asymptomatic (Asemptomatik)

Bu durumda kişi herhangi bir göğüs ağrısı yaşamaz.

Slope ST

1 - Upsloping (Yükselen eğim):

ST segmenti yukarı doğru eğimlidir. Bu genellikle normal olarak kabul edilir ve genellikle ciddi kalp problemlerine işaret etmez.

2 - Flat (Düz eğim):

ST segmenti düzdür. Bu durum, kalbin oksijen ihtiyacını karşılayamıyor olabileceğini gösterebilir ve bu nedenle potansiyel olarak anormal kabul edilir.

3 - Downsloping (Alçalan eğim):

ST segmenti aşağı doğru eğimlidir. Bu durum, kalpte ciddi bir sorun olabileceğini ve genellikle koroner arter hastalığı veya diğer ciddi kalp rahatsızlıkları ile ilişkilidir.

Thaillum

3: Normal

Bu sonuç, kalp kasının yeterli kan akışı aldığını ve herhangi bir anormallik olmadığını gösterir.

6: Orta Derecede Anormal

Bu sonuç, kalp kasının bazı bölgelerinde kan akışının yeterli olmadığını ve olası bir orta derecede iskemiyi gösterebilir.

7: Şiddetle Anormal

Bu sonuç, kalp kasının büyük bir kısmında ciddi derecede yetersiz kan akışının olduğunu ve ciddi bir kalp hastalığı riski taşıdığını gösterir.

ST depresyonunun artması genellikle kalp damarlarında tıkanıklığın veya daralmanın bir göstergesi olabilir.

Number of vessels fluro


Fluoroskopi yöntemiyle incelenen damarların sayısını ifade eder. Artması kalp rahatsızlıklarına yol açabilir.

------Kategorik Verilerin kıyası(pairplot)-----

In [ ]:
nums = [i for i in df.columns if i not in cats]

In [ ]:
sns.pairplot(df, vars=nums, hue=df.columns[-1])
plt.show()

------Her bir kategorik değişken için, o kategoriye göre sayısal değerlerin ortalama değerlerini gösteren çubuk grafikler------

In [ ]:
def plot(df, name, num, axes):
    groupby = df.groupby(name)
    mean = groupby[num].mean()
    sns.barplot(x=mean.index, y=mean, ax=axes)
    for container in axes.containers:
        axes.bar_label(container)
    axes.set_yticklabels(())
    axes.set_xticklabels(axes.get_xticklabels(), rotation=90)
    axes.set_xlabel("")
    axes.set_ylabel("")
    axes.set_title(num)

In [ ]:
for i in cats:
    display(HTML("<h2>{}</h2>".format(i)))
    fig, axes = plt.subplots(ncols=5, figsize=(15, 6))
    for j, k in enumerate(nums):
        plot(df, i, k, axes[j])
    plt.tight_layout()
    plt.show()

-------Her bir kategorik değişken için sayısal değerlerin yoğunluk tahminlemelerini (KDE plotları) ----------

In [ ]:
for i in cats[:-1]:
    display(HTML("<h2>{}</h2>".format(i)))
    fig, axes = plt.subplots(ncols=5, figsize=(15, 6))
    for j, k in enumerate(nums):
        sns.kdeplot(df, x=k, hue=i, ax=axes[j])
        axes[j].set_title(k)
        axes[j].set_ylabel("")
        axes[j].set_xlabel("")
    plt.tight_layout()
    plt.show()

--------Her bir kategorik değişken için sayısal değerlerin kutu grafikleri(box plots)------

In [ ]:
for i in cats:
    display(HTML("<h2>{}</h2>".format(i)))
    fig, axes = plt.subplots(ncols=5, figsize=(15, 6))
    for j, k in enumerate(nums):
        sns.boxplot(df, y=k, x=i, ax=axes[j])
        axes[j].set_title(k)
        axes[j].set_ylabel("")
        axes[j].set_xlabel("")
    plt.tight_layout()
    plt.show()

-------Verilerin önem derecesi-------

In [ ]:
le = LabelEncoder()
df["Heart Disease"] = le.fit_transform(df["Heart Disease"].values)
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2)
forest = RandomForestClassifier()

forest.fit(x, y)

importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

forest_importances = pd.Series(importances, index=df.columns[:-1])

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

Mean Decrease in Impurity" (MDI, Ortalama Safsızlık Azalması), karar ağaçlarının safsızlık ölçümlerinin toplam azalmasını ifade eder. Rastgele Orman modelinde, bu değer, her bir özniteliğin modelin genel performansına katkısını değerlendirir. Safsızlık genellikle Gini katsayısı veya entropi gibi ölçümlerle ifade edilir. MDI, belirli bir özniteliğin tüm ağaçlardaki dallanmalarda ne kadar safsızlık azalması sağladığının ortalamasını alır.

-------------Age - Heart Disease-----------------

In [ ]:
sns.catplot(x="Heart Disease", y="Age", hue="Sex", data=df);

-------Güven Aralığı ve Standard Sapma farkı-------

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Birinci grafik: Güven aralığı
sns.lineplot(ax=axes[0], 
             x="Age", 
             y="BP", 
             hue="Heart Disease", 
             style="Sex",
             markers=True,
             dashes=True,
             ci=100,
             data=df)
axes[0].set_title('Age vs BP by Sex (100% Confidence Interval)')

# İkinci grafik: Standart sapma
sns.lineplot(ax=axes[1], 
             x="Age", 
             y="BP", 
             hue="Heart Disease", 
             style="Sex",
             markers=True,
             dashes=True,
             ci='sd',
             data=df)
axes[1].set_title('Age vs BP by Sex (Standard Deviation)')

plt.tight_layout()
plt.show()


Güven aralığı (confidence interval), bir tahminin doğruluğu hakkında bilgi sağlar. Güven aralığı, bir parametrenin gerçek değeri hakkında bir tahmin aralığı verir.

Güven Aralığı (Confidence Interval):
Uzun Gölgeler:

Verilerin etrafındaki belirsizliğin veya varyansın yüksek olduğunu gösterir.
Modelin tahmin ettiği değerlerin geniş bir aralıkta değişebileceğini ve bu aralığın geniş olduğunu ifade eder.
Daha az güvenilir tahminler veya daha yüksek değişkenlik anlamına gelebilir.

Kısa Gölgeler:

Verilerin etrafındaki belirsizliğin veya varyansın düşük olduğunu gösterir.
Modelin tahmin ettiği değerlerin dar bir aralıkta değişebileceğini ve bu aralığın dar olduğunu ifade eder.
Daha güvenilir tahminler veya daha düşük değişkenlik anlamına gelir.

Standart Sapma Gölgeleri:

Uzun Gölgeler (Yüksek Standart Sapma):

Verilerin geniş bir aralığa yayıldığını ve ortalamadan büyük sapmalar gösterdiğini ifade eder.
Bu, veri setinin daha değişken olduğunu ve veriler arasında büyük farklılıklar bulunduğunu gösterir.
Örneğin, yaşa göre kan basıncı değişimlerinde büyük farklılıklar varsa, bu grafikteki gölgeler uzun olacaktır.

Kısa Gölgeler (Düşük Standart Sapma):

Verilerin dar bir aralığa yayıldığını ve ortalamaya yakın olduğunu ifade eder.
Bu, veri setinin daha tutarlı ve homojen olduğunu gösterir.
Örneğin, yaşa göre kolesterol seviyeleri birbirine çok yakınsa, bu grafikteki gölgeler kısa olacaktır.


---------Verilerin kıyası----------

In [ ]:
# Yaşa göre ST depresyonunu cinsiyete göre gösteren çizgi grafiği
plt.figure(figsize=(12, 6))
sns.lineplot(x='Age', y='ST depression', hue='Sex', data=df)
plt.title('Age vs ST depression by Sex')
plt.show()

# Yaşa göre damardaki lezyon sayısı ve cinsiyete göre renklendirilmiş çizgi grafiği
plt.figure(figsize=(12, 6))
sns.lineplot(x='Age', y='Number of vessels fluro', hue='Sex', data=df)
plt.title('Age vs Number of vessels fluro by Exercise Angina')
plt.show()

---------One hot encoding ve Çok kategorili dönüşüm-------

In [ ]:
columns_to_encode = ['Chest pain type', 'Thallium', 'Slope of ST', 'EKG results']
df_encoded = pd.get_dummies(df, columns=columns_to_encode)
# Her bir sütunun one-hot encoding değerlerini ve ilk 5 gözlem için göster
for column in columns_to_encode:
    print(f"{column} sütunu one-hot encoding değerleri (ilk 5 gözlem):")
    print(df_encoded[[col for col in df_encoded.columns if column in col]].head())
    print("--------------------")

# Pandas get_dummies fonksiyonu ile one-hot encoding yap
df_encoded = pd.get_dummies(df, columns=columns_to_encode)

# Sonucu göster
print(df_encoded.head())

--------Korelasyon Kıyası------

In [ ]:

numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
numeric_data = df[numeric_columns]
correlation_matrix = numeric_data.corr()

sns.heatmap(correlation_matrix, annot=True, fmt=".2f")

--------Cinsiyete göre kan basıncının kalp hastalığı durumuna göre nokta grafikleri (point plot) ile gösterildiği kategorik plot (catplot)-----------

In [ ]:
df['Heart Disease'] = df['Heart Disease'].map({1: 'Presence', 0: 'Absence'})
df['Heart Disease'] = df['Heart Disease'].astype('category')

In [ ]:
# Catplot ile cinsiyet ve egzersiz anjinasına göre kalp hastalığı durumu
sns.catplot(x="Sex", y="BP",data=df,kind = "point",hue="Heart Disease");

------------Cinsiyete göre maksimum kalp atış hızının kalp hastalığı durumuna göre kategorik olarak gösterildiği violin plot--------------

In [ ]:
# Catplot ile cinsiyet ve egzersiz anjinasına göre kalp hastalığı durumu
sns.catplot(x="Sex", y="Max HR",data=df,kind = "violin",hue="Heart Disease");

Kalp hastalığı olmayan bireylerin maksimum kalp hızları daha dar bir aralıkta yoğunlaşırken, kalp hastalığı olan bireylerin maksimum kalp hızları daha geniş bir aralıkta dağılım göstermektedir.

----------Göğüs ağrısı türlerine göre kan basıncının kalp hastalığı durumuna göre kategorik olarak gösterildiği kategorik plot (catplot)---------------

In [ ]:
sns.catplot(x="Chest pain type", y="BP",data=df,hue="Heart Disease");

----------Fluoroskopi ile incelenen damar sayısına göre kan basıncının kalp hastalığı durumuna göre kategorik olarak gösterildiği kategorik plot (catplot)---------------

In [ ]:
sns.catplot(x="Number of vessels fluro", y="BP",data=df,hue="Heart Disease");

------Kolesterol değerlerinin cinsiyete göre yoğunluk tahminlemesi (KDE plot)-----------

In [ ]:
sns.displot(df, x="Cholesterol", hue="Sex", kind="kde", height=6, aspect=1.5)

plt.title('Distribution of Cholesterol by Sex')
plt.xlabel('Cholesterol')
plt.ylabel('Density')

plt.show()

------ST eğimi değerlerinin cinsiyete göre yoğunluk tahminlemesi (KDE plot)-----------

In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x="Slope of ST", hue="Sex", fill=True, common_norm=False, alpha=0.5)

plt.title('Distribution of Slope of ST by Sex')
plt.xlabel('Slope of ST')
plt.ylabel('Density')
plt.legend(title='Sex', labels=['Female', 'Male'])
plt.show()

------Kolesterol değerleri ile kan basıncı arasındaki ilişkiyi cinsiyet ve kalp hastalığı durumlarına göre renklendirilmiş ve stille belirtilmiş bir dağılım grafiği(scatter plot)-----

In [ ]:
sns.scatterplot(data=df, x="Cholesterol", y="BP", hue="Sex", style="Heart Disease", palette="Set2")

plt.title('Scatterplot of Cholesterol vs BP')
plt.xlabel('Cholesterol')
plt.ylabel('BP')

plt.show()


------Kan basıncı ile damarların flüroskopi ile incelenen sayısı arasındaki ilişkiyi cinsiyet ve kalp hastalığı durumlarına göre gösteren ayrık dağılım grafikleri------

In [ ]:
sns.lmplot(data=df, x="BP", y="Number of vessels fluro", hue="Sex", col="Heart Disease", palette="Set2");